In [ ]:
# Recurrent Neural Network - RNN
# it uses 'memory' : past thing influences result
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
# I will use imdb dataset
from datasets import load_dataset
dataset = load_dataset("imdb")
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
# I just want to use train,test
train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
# watch example of dataset - label:0 'negative, 1 'positive'
print(train_dataset[0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
# So I need to change text to number -> use tokenizer
# BERT tokenizer(subword)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
# batch:dict(text,label), make all length of sequences same
def tokenize(batch):
  return tokenizer(batch['text'],padding=True, max_length = 256,truncation=True)

In [ ]:
# using map to tokenize
train_dataset_token = train_dataset.map(tokenize, batched = True)
test_dataset_token = test_dataset.map(tokenize, batched = True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
# new keys are made
print(train_dataset_token[0].keys())
# input_ids: sequence that token to number 101 to start, 102 to finish, 0 is padding
# attention_mask : 1:real token, 0:padding token
# token_type_dis : show the division of text -> not use in this study

dict_keys(['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])


In [ ]:
# Change the format to use torch (use only input_ids, 'attention_mask', label)
train_dataset_token.set_format('torch', columns=['input_ids','attention_mask','label'])
test_dataset_token.set_format('torch', columns=['input_ids','attention_mask','label'])

In [ ]:
print(train_dataset_token[0])

{'label': tensor(0), 'input_ids': tensor([  101,   146, 12765,   146,  6586,   140, 19556, 19368, 13329,   118,
          162, 21678,  2162, 17056,  1121,  1139,  1888,  2984,  1272,  1104,
         1155,  1103,  6392,  1115,  4405,  1122,  1165,  1122,  1108,  1148,
         1308,  1107,  2573,   119,   146,  1145,  1767,  1115,  1120,  1148,
         1122,  1108,  7842,  1118,   158,   119,   156,   119, 10148,  1191,
         1122,  1518,  1793,  1106,  3873,  1142,  1583,   117,  3335,  1217,
          170,  5442,  1104,  2441,  1737,   107,  6241,   107,   146,  1541,
         1125,  1106,  1267,  1142,  1111,  1991,   119,   133,  9304,   120,
          135,   133,  9304,   120,   135,  1109,  4928,  1110,  8663,  1213,
          170,  1685,  3619,  3362,  2377,  1417, 14960,  1150,  3349,  1106,
         3858,  1917,  1131,  1169,  1164,  1297,   119,  1130,  2440,  1131,
         3349,  1106,  2817,  1123,  2209,  1116,  1106,  1543,  1199,  3271,
         1104,  4148,  1113,  

In [ ]:
# Making dataloader
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset_token, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset_token, batch_size = 64, shuffle = True)

In [ ]:
# Vocab: the set of whole tokens. It connects word and id
print(tokenizer.vocab_size)
print(list(tokenizer.vocab)[:10])

28996
['angled', 'generate', 'evolutionary', 'lion', 'downhill', 'aspiring', '##kar', 'Company', 'Bride', 'Coleman']


In [ ]:
# Using GPU
if torch.cuda.is_available():
  device = torch.device('cuda')
  print('GPU is available')
else:
  print('GPU is not available')

GPU is available


In [ ]:
# Make simple RNN model
class RNN(nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_size, num_layers, num_classes):
    super(RNN, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embed_dim) # word embedding(28996, show the dimension of vector) -> shape(sequnce length, embedding_dimension)
    # RNN(the dimension of token vector, the size of hidden layer made by RNN, layer of RNN, True: (batch, sequence length, embedding dimension))
    self.rnn = nn.RNN(embed_dim, hidden_size, num_layers, batch_first = True)
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    x = self.embedding(x)
    x, h_n = self.rnn(x)
    h_n_last = h_n[-1] # In RNN, the results are accumulated in last layer
    out = self.fc(h_n_last)
    return out

In [ ]:
model = RNN(vocab_size = tokenizer.vocab_size, embed_dim = 128, hidden_size = 256, num_layers = 2, num_classes = 2) # num_classes -> negative or positive)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
print(model)

RNN(
  (embedding): Embedding(28996, 128)
  (rnn): RNN(128, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)


In [ ]:
# Let's go 30 -> It takes so much time....
epochs = 30
for epoch in range(1,epochs+1):
  loss_epoch = 0
  accuracy_epoch = 0

  for batch in train_loader:
    optimizer.zero_grad() # Don't forget!!
    input_ids = batch['input_ids'].to(device)
    label = batch['label'].to(device)
    outputs = model(input_ids)
    loss = criterion(outputs, label)
    loss.backward()
    optimizer.step()

    loss_epoch += loss.item()
    y_pred = outputs.argmax(dim=1)
    accuracy_epoch += (y_pred == label).sum().item()

  loss_avg = loss_epoch/len(train_loader)
  accuracy_avg = accuracy_epoch/len(train_dataset)
  if epoch % 5 == 0:
    print(f'Epoch: {epoch}, Loss: {loss_avg:10.8f}, Accuracy: {accuracy_avg:10.8f}')


Epoch: 5, Loss: 0.67784600, Accuracy: 0.55032000
Epoch: 10, Loss: 0.64756861, Accuracy: 0.58576000
Epoch: 15, Loss: 0.63088121, Accuracy: 0.60840000
Epoch: 20, Loss: 0.58650544, Accuracy: 0.64720000
Epoch: 25, Loss: 0.53643547, Accuracy: 0.66236000
Epoch: 30, Loss: 0.45968943, Accuracy: 0.70152000


In [ ]:
# Find test accuracy
accuracy = 0
loss = 0
for batch in test_loader:
  input_ids = batch['input_ids'].to(device)
  label = batch['label'].to(device)
  outputs = model(input_ids)
  y_pred = outputs.argmax(dim=1)
  loss = criterion(outputs, label)
  accuracy += (y_pred == label).sum().item()
  loss += loss.item()

accuracy_avg = accuracy/len(test_dataset)
loss_avg = loss/len(test_loader)
print(f'Test set accuracy: {accuracy_avg:10.8f}, loss: {loss_avg:10.8f}')

Test set accuracy: 0.51728000, loss: 0.00534927
